In [54]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings (not recommended)


In [55]:
# Load movie ratings dataset (replace 'ratings.csv' with your dataset)
ratings_df = pd.read_csv('C:/Users/muppi/Downloads/archive (5)/Dataset.csv')



In [56]:
ratings_df.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [57]:
ratings_df.shape

(100003, 4)

In [58]:
ratings_df

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742
...,...,...,...,...
99998,880,476,3,880175444
99999,716,204,5,879795543
100000,276,1090,1,874795795
100001,13,225,2,882399156


In [59]:
# Split data into train and test sets
X = ratings_df[['user_id', 'item_id']]
Y = ratings_df['rating']

In [60]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((90002, 2), (10001, 2), (90002,), (10001,))

In [61]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(16,activation='tanh',input_shape = X_train.shape[1:]))
model.add(Dense(32,activation='tanh'))
model.add(Dense(6,activation='softmax'))

In [62]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 16)             │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 790 (3.09 KB)

 Trainable params: 790 (3.09 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
model.compile(loss = 'sparse_categorical_crossentropy',metrics =['accuracy'],optimizer="rmsprop")

In [65]:
model.fit(X_train,Y_train,epochs=20)

Epoch 1/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3313 - loss: 1.4841
Epoch 2/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3396 - loss: 1.4627
Epoch 3/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.3375 - loss: 1.4641
Epoch 4/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3389 - loss: 1.4634
Epoch 5/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3440 - loss: 1.4648
Epoch 6/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3405 - loss: 1.4697
Epoch 7/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.3418 - loss: 1.4685
Epoch 8/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3408 - loss: 1.4697
Epoch 9/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3405 - loss: 1.4707
Epoch 10/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.3401 - loss: 1.4708
Epoch 11/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.3433 - loss: 1.4699
Epoch 12/20
2813/2813 ━━━━━━━━

In [ ]:
model.evaluate(X_test,Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3392 - loss: 1.4748


[1.4733713865280151, 0.33776623010635376]

the above is just a basic implementation of neural network which is not suitable for regression tasks 
so here i come up with advanced approach using neural collaborative filtering, which is specifically designed for recommendation tasks and often yields better results. The neural collaborative filtering approach learns low-dimensional representations of users and items (movies) and leverages their interactions to make recommendations.

here the keras components are installed for building and training neural networks

In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dense
from tensorflow.keras.callbacks import EarlyStopping

We encode user and movie IDs as numerical indices to prepare them for embedding layers in the neural network. This ensures that each user and movie is represented by a unique integer.

In [67]:
# Encode user and movie IDs
user_ids = ratings_df['user_id'].unique()
user2idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie_ids = ratings_df['item_id'].unique()
movie2idx = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}

In [68]:
ratings_df['user_id'] = ratings_df['user_id'].map(user2idx)
ratings_df['item_id'] = ratings_df['item_id'].map(movie2idx)


In [69]:
# Define embedding dimensions
embedding_dim = 50

We split the dataset into training and testing sets to evaluate the performance of the model.

In [80]:
X = ratings_df[['user_id', 'item_id']]
y = ratings_df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


We define the neural network architecture using embedding layers to learn low-dimensional representations of users and movies. We then compute the dot product of these embeddings to predict ratings.

In [70]:
# Define neural network architecture
user_input = Input(shape=(1,))
user_embedding = Embedding(input_dim=len(user_ids), output_dim=embedding_dim)(user_input)
user_embedding = Flatten()(user_embedding)

In [71]:
movie_input = Input(shape=(1,))
movie_embedding = Embedding(input_dim=len(movie_ids), output_dim=embedding_dim)(movie_input)
movie_embedding = Flatten()(movie_embedding)

In [72]:

dot_product = Dot(axes=1)([user_embedding, movie_embedding])

In [73]:
# Define model
model = Model(inputs=[user_input, movie_input], outputs=dot_product)

We compile the model using the Adam optimizer and mean squared error (MSE) loss function. MSE is suitable for regression tasks like rating prediction.

In [75]:
# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

We train the model using the training data, specifying the validation data and early stopping to prevent overfitting. We train for a maximum of 20 epochs with a batch size of 64.

In [79]:
# Train model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit([X_train['user_id'], X_train['item_id']], Y_train, 
                    validation_data=([X_test['user_id'], X_test['item_id']], Y_test),
                    epochs=20, batch_size=64, callbacks=[early_stopping])

Epoch 1/20
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6788 - val_loss: 0.8528
Epoch 2/20
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6427 - val_loss: 0.8570
Epoch 3/20
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.6119 - val_loss: 0.8622
Epoch 4/20
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.5664 - val_loss: 0.8597


We evaluate the model's performance on the test set using mean squared error (MSE). Lower MSE indicates better performance in predicting movie ratings

In [78]:
# Evaluate model
mse = model.evaluate([X_test['user_id'], X_test['item_id']], Y_test)
print("Mean Squared Error:", mse)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8356
Mean Squared Error: 0.8470625877380371
